# Comparing ground truth with prediction 3D CNN

This notebook analyses ground truth angles VS prediction angles for 3D CNN model.

## Setup
- Download and unpack training data
- Create DonkeyCar project and load configuration

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
# do we have the correct env loaded?
assert 'donkey2' == os.environ['CONDA_DEFAULT_ENV'], os.environ['CONDA_DEFAULT_ENV']

In [3]:
![ ! -d "mycar/" ] && \
donkey createcar --path mycar && \
cp config/myconfig.py mycar/

In [2]:
import sys
sys.path.insert(0, 'donkeycar')
sys.path.insert(0, 'src')


### Common settings for both directions

In [3]:
from dataloader import load_data
import numpy as np
import donkeycar as dk
from donkeycar.parts import keras
cfg = dk.load_config(config_path='mycar/config.py')

# None means all the data is used.
# use a smaller size, like 200 for testing end to end.
TUBRECORD_SIZE = None

SEQUENCE_LENGTH = 3
cfg.SEQUENCE_LENGTH = SEQUENCE_LENGTH
cfg.WANDB_ENABLED = False
cfg.TRANSFORMATIONS = ['CROP']
cfg.ROI_CROP_TOP = 60
N_FOLDS = 5
DATA_PATH = 'data'
MODELS_PATH = 'models'

MODEL_CLASS = keras.Keras3D_CNN_ModifiedOnlySteering


________             ______                   _________              
___  __ \_______________  /___________  __    __  ____/_____ ________
__  / / /  __ \_  __ \_  //_/  _ \_  / / /    _  /    _  __ `/_  ___/
_  /_/ // /_/ /  / / /  ,<  /  __/  /_/ /     / /___  / /_/ /_  /    
/_____/ \____//_/ /_//_/|_| \___/_\__, /      \____/  \__,_/ /_/     
                                 /____/                              

using donkey v4.3.5 ...



----------------------------------------------------------------------
Ran 0 tests in 0.000s

OK


loading config file: mycar/config.py
loading personal config over-rides from myconfig.py



### Load Counter Clockwise tubs and get predictions

In [ ]:
from dataloader import get_folds
import itertools

DIRECTION = 'CC'

tub_records_80_speed, tub_records_85_speed, tub_records_90_speed = load_data(cfg, DATA_PATH, DIRECTION, TUBRECORD_SIZE)

all_90_speed_data = list(itertools.chain(*list(tub_records_90_speed.values())))
train_folds, test_folds = get_folds(all_90_speed_data)

In [ ]:
from losses import ModelResults

CC_ModelResults = ModelResults(
    MODEL_CLASS,
    MODELS_PATH,
    SEQUENCE_LENGTH,
    DIRECTION,
    N_FOLDS,
    tub_records_80_speed,
    tub_records_85_speed,
    tub_records_90_speed,
    train_folds,
    test_folds,
    cfg
)

CC_ModelResults.predict_results()

CC_results = CC_ModelResults.results

In [ ]:
from collections import defaultdict
from collections import OrderedDict

def get_tub_predictions(results):
    tub_ground_truths = {}
    fold_tub_preds = defaultdict(list)
    for fold, fold_results in results.items():
        for speed_name, results in fold_results.items():
            for res in results:
                if res.name not in ['test90', 'train90']:
                    tub_ground_truths[res.name] = res.ground_truths
                    fold_tub_preds[res.name].append(res.predictions)

    # NB! Taking the average over all 5 fold's predictions
    tub_preds = OrderedDict({k: np.array(v).mean(axis=0) for k, v in sorted(fold_tub_preds.items(), key=lambda k:k[0][-2:], reverse=True)})
    return tub_preds, tub_ground_truths

CC_tub_preds, CC_tub_ground_truths = get_tub_predictions(CC_results)
len(CC_tub_preds), len(CC_tub_ground_truths)

### Plot Counter Clockwise tubs predictions and ground truths

NB:
1. Graphs need to be read from down-up direction - the start of the recording is in the bottom of the graph.
2. The .9-speed tubs have all the data in it. No train-test split is used here, the 90-speed tubs predictions are better than test and probably worse than train data.

In [ ]:
from matplotlib import pyplot as plt


def plot_gt_and_pred(tub_preds, tub_ground_truths):
    fig, axs = plt.subplots(nrows=3, ncols=2, figsize=(30, 250), sharex='all', sharey='all')
    for (name, preds), ax in zip(tub_preds.items(), axs.flat):
        ax.set(title=name, ylabel='frame number', xlabel='angle')
        ax.grid(True, axis='x')

        gt = tub_ground_truths[name]
        ax.plot(gt, range(len(gt)), 'b', label='ground truth')

        ax.plot(preds, range(len(preds)), 'g:', label='prediction')
        ax.legend()
    fig.tight_layout()
    plt.show()


plot_gt_and_pred(CC_tub_preds, CC_tub_ground_truths)


#### Thoughts
1. In these plots and even more so in the "EDA1" notebook, it can be seen that the number of laps vary greatly over different tubs and speeds. The 0.9 speed tubs contain more laps. Maybe we should calculate the mse-s for each tub/speed with each having an equal number of laps - maybe more laps is causing a higher error?
2. Slower speeds (0.8) tend to slow down quite a lot during the end of the recording. When calculating losses, these parts should be removed to get a more accurate error?

### Load Clockwise tubs and get predictions

In [ ]:
from losses import ModelResults

DIRECTION = 'CW'


tub_records_80_speed, tub_records_85_speed, tub_records_90_speed = load_data(cfg, DATA_PATH, DIRECTION, TUBRECORD_SIZE)

all_90_speed_data = list(itertools.chain(*list(tub_records_90_speed.values())))
train_folds, test_folds = get_folds(all_90_speed_data)

CW_ModelResults = ModelResults(
    MODEL_CLASS,
    MODELS_PATH,
    SEQUENCE_LENGTH,
    DIRECTION,
    N_FOLDS,
    tub_records_80_speed,
    tub_records_85_speed,
    tub_records_90_speed,
    train_folds,
    test_folds,
    cfg
)

CW_ModelResults.predict_results()


In [ ]:
CW_results = CW_ModelResults.results

CW_tub_preds, CW_tub_ground_truths = get_tub_predictions(CW_results)
len(CW_tub_preds), len(CW_tub_ground_truths)

### Plot Clockwise tubs predictions and ground truths

NB:
1. Graphs need to be read from down-up direction - the start of the recording is in the bottom of the graph.
2. The .9-speed tubs have all the data in it. No train-test split is used here, the 90-speed tubs predictions are better than test and probably worse than train data.

In [ ]:
plot_gt_and_pred(CW_tub_preds, CW_tub_ground_truths)


### Difference VS prediction and ground truth
Plot one tub's difference (prediction - ground truth) and it's predictions and ground truths graph side-by-side

In [ ]:
from collections import defaultdict

tub_diffs = defaultdict(list)
for fold, fold_results in CW_results.items():
    for speed_name, results in fold_results.items():
        for res in results:
            if res.name not in ['test90', 'train90']:
                tub_diffs[res.name].append(res.predictions - res.ground_truths)

tub_diffs = {k: np.array(v).mean(axis=0) for k,v in tub_diffs.items()}
len(tub_diffs)

In [ ]:
tub_name = '2-1-CW-80'
tub_diffs[tub_name].shape, CW_tub_preds[tub_name].shape

In [ ]:
plt.rcParams['figure.figsize'] = [30/2.54, 25/2.54]
fig, axs = plt.subplots(nrows=1, ncols=2, figsize=(30, 100), sharex='all', sharey='all')

ax1 = axs.flat[0]
ax1.set(title=tub_name + ' prediction - ground truth', ylabel='frame number', xlabel='angle')
ax1.grid(visible=True, which='both')
ax1.plot(tub_diffs[tub_name], range(len(tub_diffs[tub_name])), label='prediction - ground truth')

ax2 = axs.flat[1]
ax2.set(title=tub_name + ' predictions and ground truths', ylabel='frame number', xlabel='angle')
ax2.grid(visible=True, which='both')
ax2.plot(CW_tub_preds[tub_name], range(len(CW_tub_preds[tub_name])), 'g:', label='prediction')
gt = CW_tub_ground_truths[tub_name]
ax2.plot(gt, range(len(gt)), 'b', label='ground truth')
ax1.legend()


fig.tight_layout()
plt.show()

#### Thoughts
The difference graph is quite difficult to compare with the predictions-ground truths graphs. Maybe it is not correct to get the predictions from the 5 models and average over folds? It may make sense to train one model on all .9 speed data and look at it's predictions instead.